In [55]:
from collections import defaultdict
import random

In [124]:
def pre_processing_data(file_path):
    with open(file_path, 'r') as f:
        data = f.read()


    tokenized_sentences = []

    # split data into lines and then split each line into word_tag pairs
    lines = data.split('\n')

    for line in lines:
        word_tag_pairs = line.split()
        
        tokens = [pair.split('_') for pair in word_tag_pairs]
        
        tokenized_sentences.append(tokens)

    # Remove pontuation (when word and pos tag are the same)
    tokenized_sentences = [
        [(word, pos) for word, pos in sentence if word != pos]
        for sentence in tokenized_sentences
    ]
    

    # Replace rare words as "UNK"
    word_counts = defaultdict(int)

    for sentence in tokenized_sentences:
        for word, _ in sentence:
            word_counts[word] += 1

    UNK_THRESHOLD = 5 
    for i, sentence in enumerate(tokenized_sentences):
        tokenized_sentences[i] = [
            (word if word_counts[word] >= UNK_THRESHOLD else "UNK", pos)
            for word, pos in sentence
        ]

    # Add beginning and end of sentence tokens 
    BOS = "<BOS>"
    EOS = "<EOS>"

    tokenized_sentences = [
        [(BOS, BOS)] + sentence + [(EOS, EOS)] for sentence in tokenized_sentences
    ]


    # for sentence in tokenized_sentences:
    #     print(sentence)
    return tokenized_sentences

In [216]:
training_data = pre_processing_data('0-18-training.txt')
validation_data = pre_processing_data('19-21-development.txt')
testing_data = pre_processing_data('22-24-testing.txt')


------- Tagging

In [201]:
# Build the unigram model - attempt 1
def generate_unigram_probs(data):
    unigram_model = defaultdict(dict)

    for sentence in data:
       for word, pos in sentence:
           if pos not in unigram_model[word]:
               unigram_model[word][pos] = 1
           else:
               unigram_model[word][pos] += 1

    # Normalize probabilities
    for word, pos_counts in unigram_model.items():
       total_count = sum(pos_counts.values())
       for pos in pos_counts:
           unigram_model[word][pos] /= total_count
    return unigram_model

unigram_model = generate_unigram_probs(training_data)

print (unigram_model["like"])



{'IN': 0.7966101694915254, 'VBP': 0.046610169491525424, 'VB': 0.11652542372881355, 'JJ': 0.04025423728813559}


In [164]:
def pos_tag(sentence, unigram_model):
    tagged_sentence = []
    for word, _ in sentence:
        if word in unigram_model:
            pos_probabilities = unigram_model[word]
            
            random_value = random.uniform(0, 1)
            
            cumulative_prob = 0
            selected_pos = None
            for pos, prob in pos_probabilities.items():
                cumulative_prob += prob
                if cumulative_prob >= random_value:
                    selected_pos = pos
                    break
            
            if selected_pos:
                tagged_sentence.append((word, selected_pos))
            else:
                best_pos = max(pos_probabilities, key=pos_probabilities.get)
                tagged_sentence.append((word, best_pos))
            # best_pos = max(pos_probabilities, key=pos_probabilities.get)
            # tagged_sentence.append((word, best_pos))
        else:
            tagged_sentence.append((word, 'UNK'))
    return tagged_sentence

sentence = [('Pierre', 'NNP'), ('UNK', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('UNK', 'NNP'), ('N.V.', 'NNP')]
tagged_sentence = pos_tag(sentence, unigram_model)
print(tagged_sentence)


[('Pierre', 'NNP'), ('UNK', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('UNK', 'CD'), ('N.V.', 'NNP')]


In [166]:
def sentence_accuracy(sentence, tagged_sentence):
    correct = 0
    for (word, actual), (_, predicted) in zip(sentence, tagged_sentence):
        if actual == predicted:
            correct += 1
    return correct / len(sentence)

sentence_accuracy(sentence, tagged_sentence)

0.8571428571428571

In [192]:
# use development data to evaluate the model
def evaluate_model(data, unigram_model):
    accuracy = 0
    for sentence in data:
        tagged_sentence = pos_tag(sentence, unigram_model)
        accuracy += sentence_accuracy(sentence, tagged_sentence)
    return accuracy / len(data)
evaluate_model(validation_data, unigram_model)

0.8002105522853991

In [247]:
def generate_bigram_probs(data):
  bigram_model = defaultdict(lambda: defaultdict(int))
  bigram_probs = {}

  for sentence in data:
    prev_word = '<BOS>' # Initialize with <BOS> for each sentence
    prev_pos = '<BOS>'

    for word, pos in sentence:
      if pos not in bigram_model[(prev_word, prev_pos)].keys():
        bigram_model[(prev_word, prev_pos)][(word, pos)] = 1
      else:
        bigram_model[(prev_word, prev_pos)][(word, pos)] += 1
      prev_word = word
      prev_pos = pos

  for (prev_word, prev_pos), next_word_counts in bigram_model.items():
    total_count = sum(next_word_counts.values())
    bigram_probs[(prev_word, prev_pos)] = {next_word: count / total_count for next_word, count in next_word_counts.items()}

  return bigram_probs

bigram_model = generate_bigram_probs(training_data)
print(bigram_model['I']["want"])




KeyError: 'I'